In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.preprocessing.image import img_to_array, array_to_img

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
train_index = np.random.choice(train_images.shape[0], 600, replace = False)
train_images = train_images[train_index]
train_labels = train_labels[train_index]

test_index = np.random.choice(test_images.shape[0], 100, replace = False)
test_images = test_images[test_index]
test_labels = test_labels[test_index]

In [4]:
print(train_images.shape)
print(test_images.shape)

(600, 28, 28)
(100, 28, 28)


In [16]:
def resize_image(images, target_size = (224, 224, 3)):
  resized_img = []
  for img in images:
    img = np.stack((img, )*3, axis = -1)
    img = array_to_img(img, scale = False)
    img = img.resize(target_size[:2])
    img = img_to_array(img)
    resized_img.append(img)

  return np.array(resized_img)

In [17]:
train_images_resized = resize_image(train_images)
test_images_resized = resize_image(test_images)

In [18]:
print(f"Resized train images shape: {train_images_resized.shape}")
print(f"Resized test images shape: {test_images_resized.shape}")

Resized train images shape: (600, 224, 224, 3)
Resized test images shape: (100, 224, 224, 3)


In [19]:
train_images_resized = train_images_resized.astype('float32') / 255.0
test_images_resized = test_images_resized.astype('float32') / 255.0

In [20]:
from keras.layers import Layer

In [21]:
class LocalResponseNormalization(Layer):
  def __init__(self, k = 2, n = 5, alpha = 1e-4, beta = 0.75, **kwargs):
    super(LocalResponseNormalization, self).__init__(**kwargs)
    self.k = k
    self.n = n
    self.alpha = alpha
    self.beta = beta

  def call(self, inputs):
    return tf.nn.local_response_normalization(
        inputs,
        depth_radius = self.n,
        bias = self.k,
        alpha = self.alpha,
        beta = self.beta
    )

In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [23]:
model = Sequential()

model.add(Conv2D(
    filters=96,
    kernel_size=(11, 11),
    strides=4,
    activation='relu',
    input_shape=(224, 224, 3))
)
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(LocalResponseNormalization())

model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=1, activation='relu', padding  = "same"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(LocalResponseNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=1, activation='relu', padding = "same"))
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=1, activation='relu', padding = "same"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1, activation='relu', padding = "same"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Flatten())


model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 54, 54, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ local_response_normalization_4       │ (None, 26, 26, 96)          │               0 │
│ (LocalResponseNormalization)         │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 26, 26, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ local_response_normalization_5       │ (None, 12, 12, 256)         │               0 │
│ (LocalResponseNormalization)         │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 12, 12, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 12, 12, 384)         │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 12, 12, 256)         │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 5, 5, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4096)                │      26,218,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │          40,970 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,787,978 (178.48 MB)

 Trainable params: 46,787,978 (178.48 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [25]:
history = model.fit(
    train_images_resized,
    train_labels,
    epochs = 25,
    batch_size = 32,
    validation_data = (test_images_resized, test_labels)
)

Epoch 1/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 641ms/step - accuracy: 0.1332 - loss: 2.6624 - val_accuracy: 0.0400 - val_loss: 2.3024
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.1230 - loss: 2.2979 - val_accuracy: 0.1300 - val_loss: 2.2823
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.3664 - loss: 1.8426 - val_accuracy: 0.7000 - val_loss: 1.0192
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.7261 - loss: 0.8043 - val_accuracy: 0.7800 - val_loss: 0.6618
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.7852 - loss: 0.6282 - val_accuracy: 0.8500 - val_loss: 0.4292
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.8887 - loss: 0.2979 - val_accuracy: 0.8800 - val_loss: 0.3737
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.9346 - loss: 0.2015 - val_accuracy: 0.9000 - val_loss: 0.3689
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.9312 - loss: 0.2138 - val_accuracy: 0.8800 -